# Query Environments
This notebook demonstrates the use of the list and get functions on Managed KX Resources.


In [1]:
import os
import boto3
import json
import datetime

from managed_kx import *
from env2 import *


In [2]:
# triggers credential get
session=None

try:
    subprocess.call(["which", "ada"])
    os.system(f"ada credentials update --account={ACCOUNT_ID} --provider=isengard --role=Admin --once")
except: 
    None

if AWS_ACCESS_KEY_ID is None:
    print("Using Defaults ...")
    # create AWS session: using access variables
    session = boto3.Session()
else:
    print("Using variables ...")
    session = boto3.Session(
        aws_access_key_id=AWS_ACCESS_KEY_ID,
        aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
        aws_session_token=AWS_SESSION_TOKEN
    )

# create finspace client
client = session.client(service_name='finspace', endpoint_url=ENDPOINT_URL)

Using variables ...


# Environment

In [3]:
resp = client.get_kx_environment(environmentId=ENV_ID)
resp.pop('ResponseMetadata', None)

infra = resp.get('dedicatedServiceAccountId', "")
print(f"ID: {resp['environmentId']} ({resp['status']}): {resp['name']} ({infra})")

ID: itcdoumzc5cixt5vh4t6dp (CREATED): Managed_KX_Beta_20230530 (952604696585)


# Databases and Changesets

In [4]:
try:
    dbs = list_kx_databases(client, environmentId=ENV_ID)
except:
    dbs=[]

print(f"Environment: {ENV_ID}: Databases found: {len(dbs)}")
print()

if len(dbs) > 0:
    # Sort databases by name
    dbs = sorted(dbs, key=lambda d: d['databaseName']) 

    for d in dbs:
        db_name = d['databaseName']
        c_set_list = []
        note_str = ""

        db=client.get_kx_database(environmentId=ENV_ID, databaseName=db_name)
        db.pop('ResponseMetadata', None)

        try:
            resp = client.list_kx_changesets(environmentId=ENV_ID, databaseName=db_name)
            c_set_list = resp['kxChangesets']
        except:
            note_str = "<<Could not get changesets>>"
            break

        print("="*100)        
        print(f"Database: {db_name}, Changesets: {len(c_set_list)} {note_str}")
        print(f"{db.get('description', '')}")
        print(f"Bytes: {db['numBytes']:,} Changesets: {db['numChangesets']:,} Files: {db['numFiles']:,}")
        print("-"*100)        

        c_set_list = sorted(c_set_list, key=lambda d: d['createdTimestamp']) 

        for c in c_set_list:
            c_set_id = c['changesetId']
            print(f"Changeset ({c['status']}): {c_set_id}: Created: {c['createdTimestamp']}")
            c_rqs = client.get_kx_changeset(environmentId=ENV_ID, databaseName=db_name, changesetId=c_set_id)['changeRequests']

            chs_pdf = pd.DataFrame.from_dict(c_rqs)# .style.hide(axis='index')
            display(chs_pdf)
        print("")
                    
    print("="*100)

Environment: itcdoumzc5cixt5vh4t6dp: Databases found: 1

Database: create_delete_db, Changesets: 4 
Welcome kdb database
Bytes: 34,526,532 Changesets: 3 Files: 41
----------------------------------------------------------------------------------------------------
Changeset (FAILED): zMQ3Wtj34piBnRT4SoeDsw: Created: 2023-05-30 22:09:41.361000+00:00


,changeType,s3Path,dbPath
0,PUT,s3://kdb-demo-612841383594/data/hdb/2023.04.23/,/2023.04.23/
1,PUT,s3://kdb-demo-612841383594/data/hdb/2023.04.15/,/2023.04.15/
2,PUT,s3://kdb-demo-612841383594/data/hdb/2023.04.14/,/2023.04.14/
3,PUT,s3://kdb-demo-612841383594/data/hdb/2023.04.22/,/2023.04.22/
4,PUT,s3://kdb-demo-612841383594/data/hdb/2023.04.18/,/2023.04.18/
5,PUT,s3://kdb-demo-612841383594/data/hdb/2023.04.20/,/2023.04.20/
6,PUT,s3://kdb-demo-612841383594/data/hdb/2023.04.16/,/2023.04.16/
7,PUT,s3://kdb-demo-612841383594/data/hdb/2023.04.17/,/2023.04.17/
8,PUT,s3://kdb-demo-612841383594/data/hdb/2023.04.21/,/2023.04.21/
9,PUT,s3://kdb-demo-612841383594/data/hdb/2023.04.19/,/2023.04.19/


Changeset (COMPLETED): nsQ3XGgWIomChGVdUkzS6A: Created: 2023-05-30 22:13:05.710000+00:00


,changeType,s3Path,dbPath
0,PUT,s3://kdb-demo-612841383594/data/hdb/2023.04.23/,/2023.04.23/
1,PUT,s3://kdb-demo-612841383594/data/hdb/2023.04.15/,/2023.04.15/
2,PUT,s3://kdb-demo-612841383594/data/hdb/2023.04.14/,/2023.04.14/
3,PUT,s3://kdb-demo-612841383594/data/hdb/2023.04.22/,/2023.04.22/
4,PUT,s3://kdb-demo-612841383594/data/hdb/2023.04.18/,/2023.04.18/
5,PUT,s3://kdb-demo-612841383594/data/hdb/2023.04.20/,/2023.04.20/
6,PUT,s3://kdb-demo-612841383594/data/hdb/2023.04.16/,/2023.04.16/
7,PUT,s3://kdb-demo-612841383594/data/hdb/2023.04.17/,/2023.04.17/
8,PUT,s3://kdb-demo-612841383594/data/hdb/2023.04.21/,/2023.04.21/
9,PUT,s3://kdb-demo-612841383594/data/hdb/2023.04.19/,/2023.04.19/


Changeset (COMPLETED): 8sQ3XgXSa87YOIKc5sIgLw: Created: 2023-05-30 22:16:42.534000+00:00


,changeType,s3Path,dbPath
0,PUT,s3://kdb-demo-612841383594/data/hdb/2023.04.23/,/2023.04.23/
1,PUT,s3://kdb-demo-612841383594/data/hdb/2023.04.15/,/2023.04.15/
2,PUT,s3://kdb-demo-612841383594/data/hdb/2023.04.14/,/2023.04.14/
3,PUT,s3://kdb-demo-612841383594/data/hdb/2023.04.22/,/2023.04.22/
4,PUT,s3://kdb-demo-612841383594/data/hdb/2023.04.18/,/2023.04.18/
5,PUT,s3://kdb-demo-612841383594/data/hdb/2023.04.20/,/2023.04.20/
6,PUT,s3://kdb-demo-612841383594/data/hdb/2023.04.16/,/2023.04.16/
7,PUT,s3://kdb-demo-612841383594/data/hdb/2023.04.17/,/2023.04.17/
8,PUT,s3://kdb-demo-612841383594/data/hdb/2023.04.21/,/2023.04.21/
9,PUT,s3://kdb-demo-612841383594/data/hdb/2023.04.19/,/2023.04.19/


Changeset (COMPLETED): vsQ3bBP9NEmjohJ0F2hW5g: Created: 2023-05-30 22:47:19.805000+00:00


,changeType,s3Path,dbPath
0,PUT,s3://kdb-demo-612841383594/data/hdb/2023.04.23/,/2023.04.23/
1,PUT,s3://kdb-demo-612841383594/data/hdb/2023.04.15/,/2023.04.15/
2,PUT,s3://kdb-demo-612841383594/data/hdb/2023.04.14/,/2023.04.14/
3,PUT,s3://kdb-demo-612841383594/data/hdb/2023.04.22/,/2023.04.22/
4,PUT,s3://kdb-demo-612841383594/data/hdb/2023.04.18/,/2023.04.18/
5,PUT,s3://kdb-demo-612841383594/data/hdb/2023.04.20/,/2023.04.20/
6,PUT,s3://kdb-demo-612841383594/data/hdb/2023.04.16/,/2023.04.16/
7,PUT,s3://kdb-demo-612841383594/data/hdb/2023.04.17/,/2023.04.17/
8,PUT,s3://kdb-demo-612841383594/data/hdb/2023.04.21/,/2023.04.21/
9,PUT,s3://kdb-demo-612841383594/data/hdb/2023.04.19/,/2023.04.19/


# Clusters

## Type: HDB

In [5]:
cdf = get_clusters(client, ENV_ID, clusterType="HDB")
display(cdf)

clusterName,status,clusterType,capacityConfiguration,commandLineArguments,clusterDescription,lastModifiedTimestamp,createdTimestamp,databaseName,cacheConfigurations
cluster_create_delete_db,DELETING,HDB,"{'nodeType': 'kx.s.xlarge', 'nodeCount': 3}","[{'key': 's', 'value': '4'}, {'key': 'dbname', 'value': 'create_delete_db'}, {'key': 'codebase', 'value': 'welcomedb'}]",Demo Cluster for database create_delete_db,2023-05-30 22:48:35.185000+00:00,2023-05-30 22:17:04.188000+00:00,create_delete_db,"[{'cacheType': 'CACHE_1000', 'dbPaths': ['/']}]"


## Type: RDB

In [6]:
cdf = get_clusters(client, ENV_ID, clusterType="RDB")
display(cdf)

In [7]:
print( f"Last Run: {datetime.datetime.now()}" )

Last Run: 2023-05-30 22:49:31.430522
